# Decision Fusion Model (DFM)
Combines both sources using weighted formula: final_score = α · ECD + β · NLP
Balances neurophysiological evidence with linguistic context


Benefits of This Approach

Contextual Awareness: The model understands characters in context rather than in isolation
Error Correction: Sequential information helps correct classification errors
Improved Accuracy: Leverages both spatial (CNN) and temporal (LSTM) information
Language Integration: Better alignment with the NLP component for fusion

In [6]:
# ===============================
# Imports and Configs
# ===============================
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder
from bundle.DataCraft import load_sentence_eeg_prob_data

# ===============================
# Constants
# ===============================
NUM_CLASSES = 36
CHARACTER_MAP = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ123456789_")
MODEL_PATH = "../../model/ecd/trained_eegcnn_model_selected_channels.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SELECTED_CHANNELS = [10, 33, 48, 50, 52, 55, 59, 61]

# ===============================
# CNN Model Definition
# ===============================
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.pool = nn.AdaptiveAvgPool3d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _, _ = x.size()
        y = self.pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1, 1)
        return x * y.expand_as(x)

class EEGCNN(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(EEGCNN, self).__init__()
        self.conv1 = nn.Conv3d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm3d(32)
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm3d(64)
        self.se1 = SEBlock(64)
        self.conv3 = nn.Conv3d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm3d(128)
        self.se2 = SEBlock(128)
        self.pool = nn.AdaptiveAvgPool3d(1)
        self.fc1 = nn.Linear(128, 64)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.se1(x)
        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.se2(x)
        x = self.pool(x).squeeze()
        x = self.dropout(torch.relu(self.fc1(x)))
        return self.fc2(x)

# ===============================
# Load Data and Model
# ===============================
raw_data = load_sentence_eeg_prob_data()
if not raw_data:
    raise ValueError("Failed to load data.")

all_labels = [item["character"] for item in raw_data]
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

model = EEGCNN()
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# ===============================
# Fusion Inference with Dynamic Weights
# ===============================
correct = 0
total = 0
PRINT_LIMIT = 30

for i, sample in enumerate(raw_data):
    # --- CNN Output ---
    eeg = np.array(sample["eeg_chunk"], dtype=np.float32)[:, :, SELECTED_CHANNELS]
    eeg[30] *= 3.0
    eeg_input = torch.tensor(eeg).unsqueeze(0).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        cnn_output = model(eeg_input).squeeze()
        cnn_probs = torch.softmax(cnn_output, dim=0).cpu().numpy()

    # --- NLP Probabilities ---
    nlp_probs = np.array([sample["prob_matrix_78x64"][i][0] for i in range(36)], dtype=np.float32)

    # --- Dynamic Fusion Weights ---
    idx = sample.get("char_idx_in_sentence", 0)
    max_idx = 15  # assume sentence max ~15 chars
    idx_clamped = min(idx, max_idx)
    α = 1.0 - (idx_clamped / max_idx)   # CNN more trusted at beginning
    β = 1.0 - α                         # NLP more trusted later

    # --- Weighted Fusion ---
    final_probs = α * cnn_probs + β * nlp_probs
    predicted_index = np.argmax(final_probs)
    predicted_char = CHARACTER_MAP[predicted_index]

    true_char = sample["character"]
    if predicted_char == true_char:
        correct += 1
    total += 1

    # Optional print
    if i < PRINT_LIMIT:
        print(f"[{i}] Pred: {predicted_char} | True: {true_char} | idx_in_sentence={idx} | α={α:.2f} β={β:.2f}")

# ===============================
# Final Accuracy
# ===============================
accuracy = 100 * correct / total
print(f"\nFusion Accuracy: {accuracy:.2f}%")


Attempting to load processed data from: ../../data/sentences_eeg.pkl
Successfully loaded processed data.


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)